#Functions, correr esto es solo una vez

In [1]:
def file_names(path,file_type):
    """
    To obtain a list with the name of the files with defined extension (filetype) on a certain folder (path)

    Parameters
    ----------
    path : string
        folder name where the images are stored

    file_type : string
        extension of the files to search for (e.g. tif, png, jpg)

    Returns
    -------
    F_Names : list
        number of defined filetype files on the path folder
    """
    
    Names = []
    for name in os.listdir(path):
        name, extension = os.path.splitext(name)
        if extension == file_type and name.find("ORG") == -1:
            if (name+".tif.csv") not in os.listdir(path):
                image_name = name+extension
                Names.append(image_name)
    
    #display information
    #print(path+ '\n')

    im_count = len(Names)
    im_fnames = []

    for name in Names:
        im_fnames.append(os.path.join(path, name))

    # print('folder "' +path.split('\\')[-1]+'" = '+str(im_count) + ' files' + '\n')
    # print('file names:')
    # print(Names)
            
    return(Names, im_fnames)



In [36]:
def get_im_data(f_names):
    """
    Load image data from a list of image files (f_names).
    It doesn't supports images with different pixel dimentions
    (e.g all of them have to be 900x600 px)
    It is important for further positional correlations

    Parameters
    ----------
    f_name : string
        file name including full path where images are stored, e.g. "/folder/image_1.jpg"

    Returns
    -------
    ImsR,ImsG,ImsB: array_like
        data per channel of each image (ImsR -> matrix size = (W,H,image_count/x_frames))

    """
    
    W,H,_ = plt.imread(fnames[0]).shape      # Measure the image size based on the first image on the folder
    N = len(fnames) 
    Ims_R = np.zeros((W,H,N))
    Ims_G = np.zeros((W,H,N))
    Ims_B = np.zeros((W,H,N))
    Ims_Sum = np.zeros((W,H,N))
    
    for i in range(N):
        Im = plt.imread(f_names[0])
        Ims_R[:,:,i] = Im[:,:,0]              # Last number code the channel: 0=red, 1=green, 2=blue
        Ims_G[:,:,i] = Im[:,:,1]
        Ims_B[:,:,i] = Im[:,:,2]
        
        Chan_Sum = np.sum(Im, axis=2) #sum channels
        max_imValue = np.amax(Chan_Sum, axis = (0,1)) #get the maximum im value
        
        if max_imValue < 1: #make sure it is higher than 1 to avoid division problems
            max_imValue = 1
            
        Ims_Sum[:,:,i] = Chan_Sum*255/max_imValue #normalize and store the value
    plt.clf()
    return(Ims_R,Ims_G,Ims_B,Ims_Sum)

# at call you can take only the channels you are interested in (e.g.):
# red,_,blue=get_im_data(f_names)  ---> this only takes the red and blue channels



In [35]:
fnames[0] 

'C:\\Users\\MARIO VERA VELIZ\\Downloads\\fotos r1\\3um\\FK2021CL_(260)-Profundidad de enfoque extendida-01-Exportar imagen-10\\FK2021CL_(260)-Profundidad de enfoque extendida-01-Exportar imagen-10_m01_DAPI.tif'

In [3]:
def colony_blobs_id(data, thresh, im_name, sigma_lim=[1,10], filename='null', xlim='null', ylim='null'):
    """
    Use skimage to identify the position of each colony and define the circular region
    used by each of them

    Parameters
    ----------
    data: array of single channel image data

    thresh:
        Pixel values > thresh are included in the analysis, range (0,1)
        
    im_name:
        Name of an image on which to overlay colony positions and sizes
    
    filename: string
        filename with whom save the output image+blobs+ID
    
    sigma_lim: list
        list with lower and upper sigma limits [min_sigma, max_sigma]

    Returns
    -------
    A: array (Nx3)
        Contains the (y,x) position and size of each blob for each of N colonies detected
    """
    
    #Show thresholded image
    plt.figure(figsize=(8,8))
    plt.imshow(data>thresh)
    plt.title('Thresh Image')
    
    if xlim != 'null' and ylim != 'null':
        data2 = data[ylim[0]:ylim[1],xlim[0]:xlim[1]]
    else:
        data2 = data  #this is necesary to avoid conflicts in plot_blobs dimetions
    
    # Find the blobs
    A = skfeat.blob_log(data2, min_sigma=sigma_lim[0], max_sigma=sigma_lim[1], num_sigma=10, 
                    threshold=thresh, overlap=0.2)
    # A =[y,x,sigma]
    col_values = np.zeros((A.shape[0],A.shape[1]+2))
    col_values[:,0:3] = A
        
    radii = ((2)**(1/2))*col_values[:,2]
    col_values[:,3] = radii   # store the radii
    
    col_values[:,4]=np.arange(col_values.shape[0])  #assign the ID column
    
    #make plots
    # plot_blobs(col_values, data, ylim, xlim)
    # colony_plot(col_values,im_name, ylim, xlim)
    
    #plot the histogram of the size of the colonies
    #plt.figure()
    # plt.hist(col_values[:,3])
    # plt.title('colonies size distribution')
    # plt.ylabel('number of colonies')
    # plt.xlabel('Radius[px]')
    
    
    # if filename != 'null':
    #      plt.savefig(str(filename) + ".pdf", transparent=True)
    plt.clf()
    return(col_values)

#col_values = [x,y,sigma, radii, col_ID]



In [4]:
def plot_blobs(colonies, data, ylim='null', xlim='null'):
    
    if xlim != 'null' and ylim != 'null':
        data = data[ylim[0]:ylim[1],xlim[0]:xlim[1]]  #2 dim data --> R or G or B or sum of them
        colonies = colonies[colonies[:,1]<xlim[1],:]
        colonies = colonies[colonies[:,1]>xlim[0],:]
        colonies = colonies[colonies[:,0]<ylim[1],:]
        colonies = colonies[colonies[:,0]>ylim[0],:]
        colonies[:,1] = colonies[:,1]-xlim[0]
        colonies[:,0] = colonies[:,0]-ylim[0]
    
    xpos=colonies[:,1]
    ypos=colonies[:,0]
    #radii = ((2)**(1/2))*col_sig
    radii = colonies[:,3]
    
    fig_norm = plt.figure(figsize=(8,8))
    # plt.imshow(data, cmap='gray')
    # plt.colorbar()
    # plt.title('normalized image')
    # print('found ' + str(len(colonies))+' colonies')
    for i in range(len(colonies)):
        circle = plt.Circle((xpos[i], ypos[i]), radii[i], color='r', fill=False , 
                            lw=0.5)
        fig = plt.gcf()
        ax = fig.gca()
        ax.add_artist(circle)
    plt.clf()
    return()



In [5]:
def colony_plot(colonies,im_name, ylim, xlim):
    
    if xlim != 'null' and ylim != 'null':
        im = plt.imread(im_name)[ylim[0]:ylim[1],xlim[0]:xlim[1],:]
        colonies = colonies[colonies[:,1]<xlim[1],:]
        colonies = colonies[colonies[:,1]>xlim[0],:]
        colonies = colonies[colonies[:,0]<ylim[1],:]
        colonies = colonies[colonies[:,0]>ylim[0],:]
        colonies[:,1] = colonies[:,1]-xlim[0]
        colonies[:,0] = colonies[:,0]-ylim[0]
        
    else:
        im = plt.imread(im_name)
    
    xpos=colonies[:,1]
    ypos=colonies[:,0]
    col_sig = colonies[:,2]
    #radii = ((2)**(1/2))*col_sig
    radii = colonies[:,3]
    ID = colonies[:,4]

    fig_im = plt.figure(figsize=(8,8))
    #plt.imshow(im)
    im_name = im_name.split('\\')[-1]
    plt.title('Over "'+ im_name+'"')
    
    for i in range(len(colonies)):
        # plot the circle area identified for each colony
        #circle = plt.Circle((A[i,1], A[i,0]), 2*A[i,2], color='w', fill=False , lw=0.5)
        circle = plt.Circle((xpos[i], ypos[i]), radii[i], color='w', fill=False , lw=0.5)
        fig = plt.gcf()
        ax = fig.gca()
        ax.add_artist(circle)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        
        # attach the ID label to each colony
        plt.annotate(ID[i], xy=(xpos[i], ypos[i]), xytext=(-2, 2),
                     textcoords='offset points', ha='right', va='bottom',
                     color='white')
    plt.clf()
    return()

def smooth_data(data,sigma, im_number, row = 'middle'):

    """
    Apply gaussian filter to smooth each frame data

    Parameters
    ----------
    data: dictionary
        4 dimensional (R,G,B, and im_number) matrix with the data.
        
    sigma: double
        Filter parameter (standard deviation)

    Returns
    -------
    NSIms: dictionary
        Sum over time of Smoothed data per channel (call it nsims[channel][r,c])

    NSImsAll: array_like
        Matrix with sum of nsims over the channels (call it nsimsAll[r,c])
    
    SImsT: dictionary
        Smoothed data per channel per frame (call it as simsT[channel][r,c,f])

    """
    CHANNELS = ['R', 'G', 'B']
    NSIms = {}
    NSIms_All = np.zeros((data[CHANNELS[0]].shape[0],
                          data[CHANNELS[0]].shape[1]))
    
    NIm_All = np.zeros((NSIms_All.shape))
    
    plt.figure(figsize=(17,3))
    POS_VECT = [131,132,133]           # figure position vector
    count = 0

    for c in CHANNELS:
        # apply filter
        #Data_Sum = data[c].sum(axis=2)
        SIms = gaussian(data[c][:,:,im_number], sigma)
        
        px_dist = (SIms-SIms.min())
        max_dist =(SIms.max()-SIms.min())
        
        if max_dist < 1:    # to avoid problems of <1 division
            max_dist = 1
            
        NSIms [c] = px_dist/max_dist  # nomalization, pixel value ∈ [0,1]

        NSIms_All += NSIms[c]
        
      
        # make plot of the sum over time of smoothed data per channel
    
        # plt.subplot(POS_VECT[count])
        # plt.imshow(NSIms[c])
        # plt.colorbar()
        # plt.title(c+' channel')
    
        count += 1
    
        ##### original image normalization
        px_d2 = (SIms-SIms.min())
        max_d2 =(SIms.max()-SIms.min())
        
        if max_d2 < 1:    # to avoid problems of <1 division
            max_d2 = 1
            
        NIm = px_d2/max_d2  # nomalization, pixel value ∈ [0,1]
        NIm_All += NIm
        
    #plot the transect
    row_transect(NIm_All,'Original Image')
    row_transect(NSIms_All,'Smoothed Image')
    
    plt.clf()
    return(NSIms,NSIms_All)



In [6]:
def row_transect(data, title,  row = -1):
    """
    Plot the value of a transect (row of pixels) in a frame and plot it

    Parameters
    ----------
    data : dictionary
        dictionary with the R G B data of all images, and his names on Data['Im']

    row : int
        row where you want to see the transect
        
    title: string
        Whole figure title

    """
    
    if row == -1:
        row = int(data.shape[0]/2)
    else:
        row = int(row)  #just in case a non integer number is given
    
    plt.figure(figsize=(15,4))
    plt.subplot(121)
    plt.plot(data[row,:])
    plt.xlabel('pixels')
    plt.ylabel('value')
    plt.title('Transect')

    #plot selected line transect on the image      
    plt.subplot(122)    
    S1,S2 = data.shape
    fig = plt.gcf()
    ax = fig.gca()
    ax.imshow(data)
    rect = matplotlib.patches.Rectangle((0,row), S2, 0, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    plt.title('Sumarized image')
    
    fig.suptitle(str(title), fontsize=16)
    plt.subplots_adjust(left=0.1, wspace=0.1, top=0.8)



In [7]:
def im_zoom(x_lims, y_lims, imagen, colorbar = True):
    """
    Make a zoom of a region of interest in an image

    Parameters
    ----------
        xlims: list
            x-axis limits of the zoomed section. 
            e.g. [x_min, x_max]

        ylims: list
            y-axis limits of the zoomed section. 
            e.g. [y_min, y_max]

        imagen: numpy array
            the imagen array to be display

    """
    
    X2R = x_lims[1]-x_lims[0] #convert on steps because the rectangle patch definition
    Y2R = y_lims[1]-y_lims[0]
    
    plt.figure(figsize=(15,5))
    
    plt.subplot(121)
    #plt.imshow(imagen)
    fig = plt.gcf()
    ax = fig.gca()
    rect = matplotlib.patches.Rectangle((y_lims[0],x_lims[0]), Y2R, X2R, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    
    plt.subplot(122)
    plt.imshow(imagen[x_lims[0]:x_lims[1],y_lims[0]:y_lims[1]])
    if colorbar == True:
        plt.colorbar()



In [8]:
def binary_image(im_data, upper, lower):
    
    plt.figure(figsize=(8,8))
    plt.imshow(im_data)
    
    bin_high = im_data < upper     
          
    #if isinstance(lower,(float,int)):  #check valid input
        
    bin_low = im_data > lower
        
    #bin_grain = np.multiply(1*bin_high,1*bin_low)
    bin_grain = np.multiply(bin_high,bin_low)
        
    plt.figure(figsize=(8,8))
    #plt.imshow(bin_grain)
    
    grain_area = sum(sum(1*bin_grain))
    print('Grain area is: '+str(grain_area) + ' pixels')
    
    plt.clf()
    return(bin_grain, grain_area)



In [9]:
def Bfield_filter(colonies, bf_bin):
    filtered = []
    for i in range(len(colonies)):
        x = int(colonies[i,1])
        y = int(colonies[i,0])
        
        if bf_bin[y,x] == True:
            
            filtered.append(colonies[i,:])  
    
    return(filtered)



In [10]:
    
"""if False:
    def BF_area_filter(colonies, bf_bin):
        filtered = []
        for i in range(len(colonies)):
            x = int(colonies[i,1])
            y = int(colonies[i,0])
            r = colonies[i,3]
            
            circle = np.zeros((r,r))
            for n in r:
                for m in r:
                    if (n*n)+(m*m)<(r*r):
                        circle(n,m) = 1
            total = np.sum(np.sum(circle))
            
            #now, make a dot product between 
            
            
            if bf_bin[y,x] == True:

                filtered.append(colonies[i,:])  

        return(filtered) """




'if False:\n    def BF_area_filter(colonies, bf_bin):\n        filtered = []\n        for i in range(len(colonies)):\n            x = int(colonies[i,1])\n            y = int(colonies[i,0])\n            r = colonies[i,3]\n            \n            circle = np.zeros((r,r))\n            for n in r:\n                for m in r:\n                    if (n*n)+(m*m)<(r*r):\n                        circle(n,m) = 1\n            total = np.sum(np.sum(circle))\n            \n            #now, make a dot product between \n            \n            \n            if bf_bin[y,x] == True:\n\n                filtered.append(colonies[i,:])  \n\n        return(filtered) '

In [11]:
def save_obj(obj, name, folder, im_folder_path):
    # folder is the name of the data folder
    # name is the name of the database file
    # im_folder is used as the dictionary identifier for the experiment data.
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    database = load_obj(name, folder)
    
    im_folder = im_folder_path.split('\\')[-1]
    database[im_folder] = obj
    
    with open(folder+'/'+ name + '.pkl', 'wb') as f:
        pkl.dump(database, f, pkl.HIGHEST_PROTOCOL)
        f.close()
        print('"'+ name + '.pkl" saved on folder "'+ folder +'"')

def load_obj(name, folder ):
    try:
        with open(folder+'/' + name + '.pkl', 'rb') as f:
            return pkl.load(f)
    except FileNotFoundError:
        print('there was any previous file')
        empty_dict = {}
        return(empty_dict)

In [12]:
def imageToCsv(y, im_data, fnames, Im_ID):
    Im_ID = {}
    Im_ID['dapi'] = fnames[y]
    Im_ID['bf'] = fnames[y]

    #initialize the dictionaries to store the data
    sImS = {}
    sImSall = {}
    detected={}
    filter_sigma = {}
    threshold = {}
    sigma_lim = {}
    
    
    
    #Fill this two variables to set the filter properties
    im_num = y
    filter_sigma[im_num] = 0.4

    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    #### apply the filter
    sImS[im_num],sImSall[im_num] = smooth_data(im_data,filter_sigma[im_num],im_num)




    # perform a Zoom by slicing the image:

    # modify next values accord the section you want to see:

    x_lims = [1000,1500]    
    y_lims = [1500,2000]    

    #######################
    # Just make the plot ###
    ########################
    im_zoom(x_lims, y_lims, sImSall[im_num])




    # assign the stimated radius value in pixel units

    Rmin = 1   # --> this should be estimated based on microscope resolution and bacterial size
    Rmax = 10

    #perform the sigma limits computation:
    sigma_lim[im_num] = [Rmin/(2**0.5),Rmax/(2**0.5)]
    print('sigma limits are: ' + str(sigma_lim[im_num]))


    #define a Threshold
    threshold[im_num]=0.7 #CAMBIE ESTO PARA LOS CUPONES

    ##########################
    ## evaluate it visually ##
    ##########################

    # whole image
    thr_im = sImSall[im_num]>threshold[im_num]
    #plt.imshow(thr_im)

    # zoom
    x_lims = [1000,2000]    
    y_lims = [2000,3000]    
    im_zoom(x_lims, y_lims, thr_im, colorbar = False)

    ## Fill this two parameters only if you want to use a smaller region of the image (it's useful to verify the correct parameter choise)
    x_lims = [1000,2000]    
    y_lims = [2000,3000] 





    #########################
    ### detect the images ###
    #########################

    detected[im_num]=colony_blobs_id(sImSall[im_num], threshold[im_num], fnames[im_num], sigma_lim[im_num])#, xlim=x_lim, ylim=y_lim) 
    
    y_zoom = [500,800]
    x_zoom = [200,500]
    #colony_plot(detected[im_num], fnames[im_num], y_zoom, x_zoom)


    # image number correspont to the brighfield
    bf_num = im_data['name'].index(Im_ID['bf'])
    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    ###########################################################
    # fill this variable with desired filter parameter value:

    filter_sigma[bf_num] = 0.5

    ############################################################

    # Apply the filter
    sImS[bf_num],sImSall[bf_num] = smooth_data(im_data,filter_sigma[bf_num],bf_num)


    bf_cols = {}

    ####################################################
    ## Fill this parameters based on the image values ##

    thresh_up = 3   # grain threshold
    thresh_low = 0     # border threshold

    #####################################################

    bf_cols["bf_threshold"] = [thresh_up, thresh_low]  # save the used values
    bin_bf, grain_area = binary_image(sImSall[bf_num], thresh_up, thresh_low)

    print("image percent " + str(grain_area/bin_bf.size))

    ###################################################################

    image = 'dapi'   # write "dapi" or "dii"

    ###################################################################
    im_num = im_data['name'].index(Im_ID[image])
    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    bf_cols[im_num] =  np.asarray(Bfield_filter(detected[im_num], bin_bf))
    y_zoom = [200,600]
    x_zoom = [300,780]
    #colony_plot(bf_cols[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)
    #print('grain filtered colonies')

    #colony_plot(detected[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)

    #print('previous grain filter')

    import csv
    im_clases = ['dapi']        # They have to be in Im_ID.keys()
    Headers = ['Dye', 'Y', 'X', 'Sigma' , 'Radii', 'ID']   # Y to ID are in "detected"

    exp_name = fnames[y]   # experiment name to be create the file                            #######

    with open(exp_name+'.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                              quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(Headers)
      
        for image in im_clases:
            im_num = im_data['name'].index(Im_ID[image])
          
            for colony in bf_cols[im_num]:
                new_line = [image]
                for value in colony:
                    new_line.append(value)
                  
                filewriter.writerow(new_line)
    plt.clf()
    return

# Librerias

In [13]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os.path
import glob
import pickle as pkl
import time
import pandas as pd
from glob import glob
import gc

#modify some matplotlib parameters to manage the images for illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from skimage.filters import gaussian
import skimage.feature as skfeat
from scipy import ndimage

# Upload folder

In [ ]:
#Clone github
!git clone https://github.com/OSOMONOSO/Image_analysis_folders
MainFolder = './Image_analysis_folders'

fatal: destination path 'Image_analysis_folders' already exists and is not an empty directory.


In [ ]:
MainFolder = './Image_analysis_folders'

#Upload foders drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
#from google.colab import files

#MainFolder ='/content/gdrive/MyDrive/fotos r1/46um' # 3um, 15um, 46um, Org
MainFolder ='fotos r1/3um' # 3um, 15um, 46um, Org

data = os.listdir(MainFolder)

# Ciclo for para imagenes dentro de una carpeta

In [ ]:
#Delete all folders
#%rm -rf Image_analysis_folders

In [37]:
#Upload folder and detect where it is
data = os.listdir(MainFolder)
df = pd.DataFrame(data, columns=['Folder'])
df.sort_values(by=['Folder'], inplace=True)
df = df.iloc[0::, 0] #[1::, 0] si la primera imagen está defectuosa
df = df.reset_index(drop=True)

largo_data = df.count()

im_extension = '.tif'

for x in range(largo_data):

  nested_folder = (MainFolder + "/" + df[x]+"/")
  fpath = os.path.abspath(nested_folder)
  im_names, fnames = file_names(fpath, im_extension)
  Im_ID = {}
  im_data = {}
  if len(fnames) == 0:
    print("Nada que procesar en el fichero:", nested_folder)
  else:
    print(len(fnames))
    im_data['R'], im_data['G'], im_data['B'], im_data['NormSum'] = get_im_data(fnames)
    im_data['name'] = fnames
    # ID bright and dapi
    f_categories = list(Im_ID.keys())
    for cat in f_categories:
      print(str(cat) + ' file is: \n' + str(Im_ID[cat]) + '\n')
    for y in range(0, len(fnames)):
        print(y, "de ", len(fnames))
        # to store the related image source
        # also define a vector with the channels
        imageToCsv(y, im_data, fnames, Im_ID)
        gc.collect()


16


ValueError: not enough values to unpack (expected 3, got 0)

In [23]:
fnames[1]

'C:\\Users\\MARIO VERA VELIZ\\Downloads\\fotos r1\\3um\\FK2021CL_(260)-Profundidad de enfoque extendida-01-Exportar imagen-10\\FK2021CL_(260)-Profundidad de enfoque extendida-01-Exportar imagen-10_m02_DAPI.tif'

In [ ]:
#Upload folder and detect where it is
data = os.listdir(MainFolder)
df = pd.DataFrame(data, columns=['Folder'])
df.sort_values(by=['Folder'], inplace=True)
df = df.iloc[1::,0]
df = df.reset_index(drop=True)

largo_data = df.count()

im_extension = 'DAPI.tif'
for x in range(largo_data): 
  nested_folder = ( MainFolder + "/" + df[x])
  fpath = os.path.abspath(nested_folder)
  im_names, fnames = file_names(fpath,im_extension)

  for x in range (largo_data):

    #ID bright and dapi
    Im_ID = {}
    Im_ID['dapi'] = fnames[x]
    Im_ID['bf'] = fnames[x]

    f_categories = list(Im_ID.keys())
    for cat in f_categories:
        print(str(cat) + ' file is: \n'+ str(Im_ID[cat]) +'\n' )

    
    
    # to store the related image source
    # also define a vector with the channels
    im_data={}
    im_data['R'],im_data['G'],im_data['B'],im_data['NormSum'] = get_im_data(fnames)
    im_data['name']=fnames    
    CHANNELS=['R','G','B']


    
    #initialize the dictionaries to store the data
    sImS = {}
    sImSall = {}
    detected={}
    filter_sigma = {}
    threshold = {}
    sigma_lim = {}
    
    
    
    #Fill this two variables to set the filter properties
    im_num = x
    filter_sigma[im_num] = 0.4

    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    #### apply the filter
    sImS[im_num],sImSall[im_num] = smooth_data(im_data,filter_sigma[im_num],im_num)




    # perform a Zoom by slicing the image:

    # modify next values accord the section you want to see:

    x_lims = [1000,1500]    
    y_lims = [1500,2000]    

    #######################
    # Just make the plot ###
    ########################
    im_zoom(x_lims, y_lims, sImSall[im_num])




    # assign the stimated radius value in pixel units

    Rmin = 1   # --> this should be estimated based on microscope resolution and bacterial size
    Rmax = 10

    #perform the sigma limits computation:
    sigma_lim[im_num] = [Rmin/(2**0.5),Rmax/(2**0.5)]
    print('sigma limits are: ' + str(sigma_lim[im_num]))


    #define a Threshold
    threshold[im_num]=0.7 #CAMBIE ESTO PARA LOS CUPONES

    ##########################
    ## evaluate it visually ##
    ##########################

    # whole image
    thr_im = sImSall[im_num]>threshold[im_num]
    #plt.imshow(thr_im)

    # zoom
    x_lims = [1000,2000]    
    y_lims = [2000,3000]    
    im_zoom(x_lims, y_lims, thr_im, colorbar = False)

    ## Fill this two parameters only if you want to use a smaller region of the image (it's useful to verify the correct parameter choise)
    x_lims = [1000,2000]    
    y_lims = [2000,3000] 





    #########################
    ### detect the images ###
    #########################

    detected[im_num]=colony_blobs_id(sImSall[im_num], threshold[im_num], fnames[im_num], sigma_lim[im_num])#, xlim=x_lim, ylim=y_lim) 
    
    y_zoom = [500,800]
    x_zoom = [200,500]
    colony_plot(detected[im_num], fnames[im_num], y_zoom, x_zoom)


    # image number correspont to the brighfield
    bf_num = im_data['name'].index(Im_ID['bf'])
    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    ###########################################################
    # fill this variable with desired filter parameter value:

    filter_sigma[bf_num] = 0.5

    ############################################################

    # Apply the filter
    sImS[bf_num],sImSall[bf_num] = smooth_data(im_data,filter_sigma[bf_num],bf_num)


    bf_cols = {}

    ####################################################
    ## Fill this parameters based on the image values ##

    thresh_up = 3   # grain threshold
    thresh_low = 0     # border threshold

    #####################################################

    bf_cols["bf_threshold"] = [thresh_up, thresh_low]  # save the used values
    bin_bf, grain_area = binary_image(sImSall[bf_num], thresh_up, thresh_low)

    print("image percent " + str(grain_area/bin_bf.size))

    ###################################################################

    image = 'dapi'   # write "dapi" or "dii"

    ###################################################################
    im_num = im_data['name'].index(Im_ID[image])
    print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

    bf_cols[im_num] =  np.asarray(Bfield_filter(detected[im_num], bin_bf))
    y_zoom = [200,600]
    x_zoom = [300,780]
    #colony_plot(bf_cols[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)
    #print('grain filtered colonies')

    #colony_plot(detected[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)

    #print('previous grain filter')

    import csv
    im_clases = ['dapi']        # They have to be in Im_ID.keys()
    Headers = ['Dye', 'Y', 'X', 'Sigma' , 'Radii', 'ID']   # Y to ID are in "detected"

    exp_name = fnames[x]   # experiment name to be create the file                            #######

    with open(exp_name+'.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                              quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(Headers)
      
        for image in im_clases:
            im_num = im_data['name'].index(Im_ID[image])
          
            for colony in bf_cols[im_num]:
                new_line = [image]
                for value in colony:
                    new_line.append(value)
                  
                filewriter.writerow(new_line)


  #metada

  """
  m_headers = ['Image','Parameter', 'Value']

  s_min = {}
  s_max = {}

  for i in list(sigma_lim.keys()):
      s_min[i] = sigma_lim[i][0]
      s_max[i] = sigma_lim[i][1]

  param_values = {
      'smooth' : filter_sigma,
      'threshold': threshold,
      'sigma_min': s_min,
      'sigma_max': s_max
  } 

  parameters = list(param_values.keys())

  with open("metadata_"+exp_name+'.csv', 'w') as csvfile:
      filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
      filewriter.writerow(m_headers)
    
      for image in im_clases:
          im_num = im_data['name'].index(Im_ID[image])
        
          for param in parameters:
              new_line = []
              new_line.append(Im_ID[image])            #add image name
              new_line.append(param)                   #add param name 
              new_line.append(param_values[param][im_num])   #add image name
                
              filewriter.writerow(new_line)"""


  fpath.split('\\')[-1]

  ### Organize the information, it is not necessary to modify anything here.
  filtered_images = {}
  filtered_images['images'] = sImS
  filtered_images['filter_sigma'] = filter_sigma

  images_data = {}
  images_data["smooth"] = filter_sigma
  images_data["detection"] = {}
  images_data["detection"]['colonies'] = detected
  images_data["detection"]['threshold'] = threshold
  images_data["detection"]['sigma_domain'] = sigma_lim
  images_data["detection"]['bf_colonies'] = bf_cols

  #Save the data
  save_obj(images_data, 'database', 'data', fpath)

  data2 = load_obj('database', 'data')

  data2.keys()



# Sección nueva

In [ ]:
#%cd ..
!pwd

In [ ]:
#concatenar todos los archivos .csv


data = os.listdir(MainFolder)
df2 = pd.DataFrame(data, columns=['Folder'])
df2.sort_values(by=['Folder'], inplace=True)
df2 = df2.iloc[1::,0]
df2 = df2.reset_index(drop=True)
largo_data = df2.count()
file_extension = '.csv'


#by folders
All_colonies=pd.DataFrame()
for x in range(largo_data): 
  csv_files = sorted(glob(MainFolder+'/' + df2[x] + '/*' + file_extension))
  df = pd.concat((pd.read_csv(file).assign(filename = file) 
  for file in csv_files), ignore_index = False) #False si quiere mantener el index, True si no
  df.to_csv(df2[x]+'.csv') #un archivo por carpeta
  All_colonies = All_colonies.append(df)
All_colonies.to_csv('All_colonies_positions.csv') #un archivo para todas

In [ ]:
#now, for summary

data = os.listdir(MainFolder)
df2 = pd.DataFrame(data, columns=[ 'Folder'])
df2.sort_values(by=['Folder'], inplace=True)
df2 = df2.iloc[1::,0]
df2 = df2.reset_index(drop=True)
largo_data = df2.count()
file_extension = '.csv'
#by folders
All_colonies=pd.DataFrame()
for x in range(largo_data): 
  csv_files = sorted(glob(MainFolder+'/' + df2[x] + '/*' + file_extension))
  df = pd.concat((pd.read_csv(file).assign(filename = file).iloc[-1:]
                  for file in csv_files), ignore_index = False)
  
df = df.reset_index().rename(columns={'index':'Count'})
df['Count']=df['Count']+1 #agregarle 1 para que quede igual
df

#Zona de prueb de Luna

In [ ]:
#Delete all folders
#%rm -rf Image_analysis_folders
%whos #show all active variables

In [ ]:
#Image_analysis como funcion
def Image_analysis(x,fnames,im_names):
#ID bright and dapi
  Im_ID = {}
  Im_ID['dapi'] = fnames[x]
  Im_ID['bf'] = fnames[x]

  f_categories = list(Im_ID.keys())
  for cat in f_categories:
      print(str(cat) + ' file is: \n'+ str(Im_ID[cat]) +'\n' )
  # to store the related image source
  # also define a vector with the channels
  im_data={}
  im_data['R'],im_data['G'],im_data['B'],im_data['NormSum'] = get_im_data(fnames)
  im_data['name']=fnames    
  CHANNELS = ['R','G','B']

  #initialize the dictionaries to store the data
  sImS = {}
  sImSall = {}
  detected={}
  filter_sigma = {}
  threshold = {}
  sigma_lim = {}
  
  #Fill this two variables to set the filter properties
  im_num = x
  filter_sigma[im_num] = 0.4

  print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

  #### apply the filter
  sImS[im_num],sImSall[im_num] = smooth_data(im_data,filter_sigma[im_num],im_num)

  # perform a Zoom by slicing the image:

  # modify next values accord the section you want to see:

  x_lims = [1000,1500]    
  y_lims = [1500,2000]    

  #######################
  # Just make the plot ###
  ########################
  im_zoom(x_lims, y_lims, sImSall[im_num])

  # assign the stimated radius value in pixel units

  Rmin = 1   # --> this should be estimated based on microscope resolution and bacterial size
  Rmax = 10

  #perform the sigma limits computation:
  sigma_lim[im_num] = [Rmin/(2**0.5),Rmax/(2**0.5)]
  print('sigma limits are: ' + str(sigma_lim[im_num]))

  #define a Threshold
  threshold[im_num]=0.7 #CAMBIE ESTO PARA LOS CUPONES

  ##########################
  ## evaluate it visually ##
  ##########################

  # whole image
  thr_im = sImSall[im_num]>threshold[im_num]
  #plt.imshow(thr_im)

  # zoom
  x_lims = [1000,2000]    
  y_lims = [2000,3000]    
  im_zoom(x_lims, y_lims, thr_im, colorbar = False)

  ## Fill this two parameters only if you want to use a smaller region of the image (it's useful to verify the correct parameter choise)
  x_lims = [1000,2000]    
  y_lims = [2000,3000] 

  #########################
  ### detect the images ###
  #########################

  detected[im_num]=colony_blobs_id(sImSall[im_num], threshold[im_num], fnames[im_num], sigma_lim[im_num])#, xlim=x_lim, ylim=y_lim) 
  
  y_zoom = [500,800]
  x_zoom = [200,500]
  colony_plot(detected[im_num], fnames[im_num], y_zoom, x_zoom)

  # image number correspont to the brighfield
  bf_num = im_data['name'].index(Im_ID['bf'])
  print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

  ###########################################################
  # fill this variable with desired filter parameter value:

  filter_sigma[bf_num] = 0.5
  ############################################################

  # Apply the filter
  sImS[bf_num],sImSall[bf_num] = smooth_data(im_data,filter_sigma[bf_num],bf_num)


  bf_cols = {}

  ####################################################
  ## Fill this parameters based on the image values ##

  thresh_up = 3   # grain threshold
  thresh_low = 0     # border threshold

  #####################################################

  bf_cols["bf_threshold"] = [thresh_up, thresh_low]  # save the used values
  bin_bf, grain_area = binary_image(sImSall[bf_num], thresh_up, thresh_low)

  print("image percent " + str(grain_area/bin_bf.size))

  ###################################################################

  image = 'dapi'   # write "dapi" or "dii"

  ###################################################################
  im_num = im_data['name'].index(Im_ID[image])
  print('working on "' + im_data['name'][im_num].split("\\")[-1]+ '"')

  bf_cols[im_num] =  np.asarray(Bfield_filter(detected[im_num], bin_bf))
  y_zoom = [200,600]
  x_zoom = [300,780]
  #colony_plot(bf_cols[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)
  #print('grain filtered colonies')

  #colony_plot(detected[im_num], fnames[im_num], ylim = y_zoom, xlim = x_zoom)

  #print('previous grain filter')
  return(im_data, bf_cols)

In [ ]:
##tellme how many .csv are in each folder ##yes, I just reused the code
data = os.listdir(MainFolder)
df = pd.DataFrame(data, columns=[ 'Folder'])
df.sort_values(by=['Folder'], inplace=True)
df = df.iloc[1::,0]
df = df.reset_index(drop=True)
largo_data = df.count()

for y in range(largo_data): 
  nested_folder = (MainFolder + "/" + df[y])
  fpath = os.path.abspath(nested_folder)
  im_names, fnames = file_names(fpath,im_extension)
  fnamesCSV = []
  fnamesCSV = [x for x in fnames if x.endswith('.csv')] ############### to filter out folders with .csv
  how_long = len(fnamesCSV)
  how_long = str(how_long)
  print(df[y]+' has '+how_long+'.csv files')

In [ ]:
#Para que corra Image_analysis!
#Edition so it wont run folders with .csv
data = os.listdir(MainFolder)
df = pd.DataFrame(data, columns=[ 'Folder'])
df.sort_values(by=['Folder'], inplace=True)
df = df.iloc[1::,0]
df = df.reset_index(drop=True)
largo_data = df.count()
for y in range(largo_data): 
  nested_folder = (MainFolder + "/" + df[y])
  fpath = os.path.abspath(nested_folder)
  im_names, fnames = file_names(fpath,im_extension)
  CHANNELS = ['R','G','B'] ########
  fnamesCSV = []
  fnamesCSV = [x for x in fnames if x.endswith('.csv')] ############### to filter out folders with .csv
  if len(fnamesCSV) == 16:
    print(df[y]+' already has .csv files')
  else:
    for x in range (16):
      plt.close('all')  ###a ver si funciona :)
      Im = np.array([])  ############## trying to delete/empty arrays so it wont use all the ram
      im = np.array([])  ############### up
      im_data, bf_cols = Image_analysis(x,fnames,im_names)
      Im_ID = {}
      Im_ID['dapi'] = fnames[x]
      Im_ID['bf'] = fnames[x]
      im_clases = ['dapi']        # They have to be in Im_ID.keys()
      Headers = ['Dye', 'Y', 'X', 'Sigma' , 'Radii', 'ID']   # Y to ID are in "detected"
      exp_name = fnames[x]   # experiment name to be create the file                            #######
      with open(exp_name+'.csv', 'w') as csvfile:
          filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
          filewriter.writerow(Headers)
          for image in im_clases:
              im_num = im_data['name'].index(Im_ID[image])
              for colony in bf_cols[im_num]:
                  new_line = [image]
                  for value in colony:
                      new_line.append(value)
                    
                  filewriter.writerow(new_line)
plt.close('all')  ###a ver si funciona :)
